In [ ]:
#received reapeatMasker output from Eric Rondeau for the S.salar genome release 100
# it's on Gannet here: https://gannet.fish.washington.edu/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff
# Eric also sent a file that contains TE annotations for sequence IDs in the RepeatMasker file
# it's on Gannet here: https://gannet.fish.washington.edu/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.txt


#I want to generate a TE.gff file that I can map the DMRs to so I need to add the TE annotations to the gff file 
#Then combine it with the other gffs I have and run intersectBed with the DMRs

In [ ]:
#preview the gff

In [1]:
!head /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff

##gff-version 2
##date 2017-05-23
##sequence-region genome_seq.fa
gi|924926424|ref|NW_012561447.1|	RepeatMasker	similarity	1	697	 1.3	+	.	Target "Motif:Ssa_0528" 978 1675
gi|924926424|ref|NW_012561447.1|	RepeatMasker	similarity	690	1060	 3.2	+	.	Target "Motif:Ssa_1374" 526 896
gi|924926427|ref|NW_012416322.1|	RepeatMasker	similarity	1	1687	 4.4	-	.	Target "Motif:Ssa_0994" 1 1681
gi|924926429|ref|NW_012473295.1|	RepeatMasker	similarity	4	1310	 2.7	-	.	Target "Motif:Ssa_1037" 1 1311
gi|924926430|ref|NW_012550955.1|	RepeatMasker	similarity	1	1173	 7.8	+	.	Target "Motif:Ssa_1062" 1 1174
gi|924926431|ref|NW_012554585.1|	RepeatMasker	similarity	1	1123	26.7	+	.	Target "Motif:Ssa_0509" 1 1115
gi|924926432|ref|NW_012543257.1|	RepeatMasker	similarity	1	1204	 2.8	-	.	Target "Motif:Ssa_1037" 1 1217


In [3]:
!tail /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff

gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038386	159038431	19.6	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038420	159038465	10.9	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038454	159038499	15.2	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038488	159038533	15.2	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038522	159038567	10.9	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038556	159038601	15.2	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038590	159038635	13.0	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038624	159038669	10.9	+	.	Target "Motif:Ssa_0367" 3001 3046
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038658	

In [ ]:
#preview the txt file

In [2]:
!head /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.txt

In [ ]:
#I need to map the "SSa_XXXX" IDs in the txt file to the "Motif:Ssa_XXXX" in the gff
#First find out what field of the gff the ssa id is in

In [8]:
!awk -F"\t" '{print $9}' /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff |\
head




Target "Motif:Ssa_0528" 978 1675
Target "Motif:Ssa_1374" 526 896
Target "Motif:Ssa_0994" 1 1681
Target "Motif:Ssa_1037" 1 1311
Target "Motif:Ssa_1062" 1 1174
Target "Motif:Ssa_0509" 1 1115
Target "Motif:Ssa_1037" 1 1217
awk: write error on /dev/stdout
 input record number 1056, file /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff
 source line number 1


In [ ]:
#remove everything before and after the quotes

In [10]:
!tail /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff |\
awk -F"\t" '{OFS=FS="\t"}{sub(/Target "Motif:/,"",$9);sub(/" .*/,"",$9);print $9}' 

Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367
Ssa_0367


In [11]:
!tail /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff |\
awk -F"\t" '{OFS=FS="\t"}{sub(/Target "Motif:/,"",$9);sub(/" .*/,"",$9);print $0}' 

gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038386	159038431	19.6	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038420	159038465	10.9	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038454	159038499	15.2	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038488	159038533	15.2	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038522	159038567	10.9	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038556	159038601	15.2	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038590	159038635	13.0	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038624	159038669	10.9	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038658	159038703	10.9	+	.	Ssa_0367
gi|925216783|ref|NC_027300.1|	RepeatMasker	similarity	159038692	159038737	10.9	+	.	Ssa_0367


In [13]:
!tail /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff |\
awk -F"\t" '{OFS=FS="\t"}{sub(/Target "Motif:/,"",$9);sub(/" .*/,"",$9);print $0}' |\
awk -F"\t" 'NR==FNR{a[$1]=$2;next}{print $0" "a[$9]}' \
/Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.txt \
-

In [ ]:
#it seems like it will work so apply code to entire gff

In [14]:
!awk -F"\t" '{OFS=FS="\t"}{sub(/Target "Motif:/,"",$9);sub(/" .*/,"",$9);print $0}' \
/Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1_vs_ncbi_release_100.genome_seq.fa.out.gff|\
awk -F"\t" 'NR==FNR{a[$1]=$2;next}{print $0" "a[$9]}' \
/Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.txt \
- \
> /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.TEannotations.gff

In [15]:
!head -25 /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.TEannotations.gff

##gff-version 2 
##date 2017-05-23 
##sequence-region genome_seq.fa 














gi|924926434|ref|NW_012367147.1|	RepeatMasker	similarity	1306	1360	19.3	+	.	A-rich 



gi|924926434|ref|NW_012367147.1|	RepeatMasker	similarity	4823	4849	 0.0	+	.	(CT)n 

gi|924926435|ref|NW_012355637.1|	RepeatMasker	similarity	2332	2388	 3.8	+	.	(CTC)n 
gi|924926435|ref|NW_012355637.1|	RepeatMasker	similarity	2419	2510	16.4	+	.	(CTCCT)n 


In [ ]:
#looks good; now i want to remove the NW sequence

In [16]:
!cat /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.TEannotations.gff |\
awk -F"\t" '{if($0!~/NW_/)print $0}' \
> /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.TEannotations.noNW.gff

In [ ]:
#check TE categories

In [18]:
!cat /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.TEannotations.noNW.gff |\
awk -F"\t" '{print $9}' |\
awk -F" " '{if($1!~/Ssa_/)print $1;else print $2}' |\
sort | uniq -c

   3 
2839 (A)n
   1 (AAAAAAG)n
   4 (AAAAAAT)n
   3 (AAAAAC)n
   1 (AAAAACA)n
   4 (AAAAAG)n
   1 (AAAAAGA)n
   9 (AAAAAT)n
   1 (AAAAATC)n
  27 (AAAAC)n
   2 (AAAACA)n
   1 (AAAACAA)n
   1 (AAAACAGA)n
   1 (AAAACAGTCTC)n
   1 (AAAACC)n
   1 (AAAACT)n
   1 (AAAACTT)n
  10 (AAAAG)n
   1 (AAAAGA)n
   1 (AAAAGAAAG)n
   2 (AAAAGAT)n
   1 (AAAAGC)n
   1 (AAAAGG)n
   1 (AAAAGTAA)n
  53 (AAAAT)n
   1 (AAAATAAAT)n
   1 (AAAATAC)n
   1 (AAAATACGC)n
   1 (AAAATAG)n
   1 (AAAATAGTA)n
   3 (AAAATG)n
   1 (AAAATGT)n
   2 (AAAATT)n
 222 (AAAC)n
  18 (AAACA)n
   2 (AAACAA)n
   1 (AAACAAAA)n
   1 (AAACAAT)n
   9 (AAACAC)n
   1 (AAACACA)n
   1 (AAACACACAG)n
   1 (AAACACAGGT)n
   2 (AAACACTG)n
   2 (AAACAG)n
   1 (AAACAGCT)n
   1 (AAACATAACT)n
   1 (AAACATAGTG)n
   1 (AAACC)n
   1 (AAACCACTAG)n
   1 (AAACCATGTG)n
   4 (AAACT)n
   2 (AAACTA)n
   1 (AAACTACCAT)n
   1 (AAACTG)n
   1 (AAACTGT)n
   2 (AAACTTG)n
 262 (AAAG)n
  14 (AAAGA)n
   1 (AAAGAA)n
   1 (AAAGAATG)n
   1 (AAAGACAACT)n
   1 (AAAGACTA)n
  

63055 (AC)n
 675 (ACA)n
 354 (ACAA)n
  11 (ACAAA)n
   1 (ACAAAACG)n
   1 (ACAAAATAC)n
  16 (ACAAAC)n
   1 (ACAAACAAT)n
   2 (ACAAACAC)n
   1 (ACAAACAG)n
   1 (ACAAAG)n
   1 (ACAAAGC)n
   1 (ACAAAT)n
   1 (ACAAATA)n
  95 (ACAAC)n
  16 (ACAACA)n
   6 (ACAACAA)n
  11 (ACAACAC)n
   1 (ACAACACAGC)n
   1 (ACAACACT)n
   1 (ACAACAG)n
   1 (ACAACAGG)n
   5 (ACAACAT)n
   1 (ACAACATG)n
   1 (ACAACATGGA)n
  19 (ACAACC)n
   1 (ACAACCAGGC)n
   1 (ACAACCATA)n
   1 (ACAACCTG)n
   1 (ACAACG)n
   1 (ACAACGC)n
   8 (ACAACT)n
   1 (ACAACTAG)n
   1 (ACAACTAT)n
   2 (ACAACTC)n
   1 (ACAACTG)n
  22 (ACAAG)n
  11 (ACAAGA)n
   4 (ACAAGAA)n
   2 (ACAAGAC)n
   2 (ACAAGAT)n
   1 (ACAAGATGTT)n
  10 (ACAAGC)n
   1 (ACAAGCA)n
   1 (ACAAGCT)n
   3 (ACAAGG)n
   2 (ACAAGGA)n
   1 (ACAAGGCATT)n
   1 (ACAAGTA)n
   1 (ACAAGTG)n
 114 (ACAAT)n
  25 (ACAATA)n
   6 (ACAATAA)n
   9 (ACAATAC)n
   1 (ACAATACATT)n
   3 (ACAATAG)n
   5 (ACAATAT)n
  10 (ACAATC)n
   5 (ACAATCA)n
   2 (ACAATCTA)n
  11 (ACAATG)n
   1 (ACAATGA)n
   1 (

  40 (AGAGGG)n
   1 (AGAGGGA)n
   1 (AGAGGGACAC)n
   2 (AGAGGGAG)n
   1 (AGAGGGAGGT)n
   2 (AGAGGGAT)n
   1 (AGAGGGATGG)n
   1 (AGAGGGC)n
   4 (AGAGGGG)n
   1 (AGAGGGGAG)n
   1 (AGAGGGGGAA)n
  39 (AGAGGT)n
   1 (AGAGGTAACT)n
   2 (AGAGGTAG)n
   2 (AGAGGTAGAG)n
  32 (AGAGGTC)n
   1 (AGAGGTCA)n
   1 (AGAGGTCAG)n
   1 (AGAGGTCGAG)n
  12 (AGAGGTG)n
   1 (AGAGGTGAG)n
   1 (AGAGGTGG)n
   1 (AGAGGTGGC)n
  25 (AGAGGTT)n
   1 (AGAGGTTAAG)n
   1 (AGAGGTTC)n
   1 (AGAGGTTTAG)n
  56 (AGAGT)n
   6 (AGAGTA)n
   2 (AGAGTAA)n
   7 (AGAGTAG)n
   1 (AGAGTAGACT)n
   1 (AGAGTAGAGC)n
   1 (AGAGTAGAGT)n
   1 (AGAGTAGCAG)n
   1 (AGAGTAT)n
   1 (AGAGTATC)n
   7 (AGAGTC)n
   2 (AGAGTCA)n
   2 (AGAGTCAC)n
   1 (AGAGTCAGGA)n
   1 (AGAGTCT)n
  39 (AGAGTG)n
   2 (AGAGTGA)n
   1 (AGAGTGACAG)n
   4 (AGAGTGAG)n
   1 (AGAGTGC)n
   9 (AGAGTGG)n
   1 (AGAGTGT)n
   1 (AGAGTGTGT)n
   6 (AGAGTT)n
   1 (AGAGTTACAG)n
   1 (AGAGTTAG)n
   1 (AGAGTTAGAG)n
   2 (AGAGTTC)n
   1 (AGAGTTCT)n
   1 (AGAGTTTACT)n
   1 (AGAGTTTG)n
 415

  93 (ATGTGT)n
   3 (ATGTGTA)n
   4 (ATGTGTC)n
   1 (ATGTGTCTGA)n
   3 (ATGTGTG)n
   1 (ATGTGTGC)n
   1 (ATGTGTGCT)n
   2 (ATGTGTGT)n
   1 (ATGTGTGTA)n
   2 (ATGTGTGTAT)n
   2 (ATGTGTT)n
  46 (ATGTT)n
  13 (ATGTTA)n
   3 (ATGTTAA)n
   3 (ATGTTAC)n
   1 (ATGTTACC)n
   1 (ATGTTAG)n
   4 (ATGTTAT)n
   1 (ATGTTATCAC)n
   1 (ATGTTATGAC)n
   3 (ATGTTATGG)n
   8 (ATGTTC)n
   1 (ATGTTCA)n
   1 (ATGTTCCCT)n
   2 (ATGTTCT)n
   1 (ATGTTCTA)n
  12 (ATGTTG)n
   2 (ATGTTGC)n
   3 (ATGTTGT)n
   1 (ATGTTGTGA)n
   1 (ATGTTGTTC)n
  32 (ATGTTT)n
   3 (ATGTTTA)n
   1 (ATGTTTACC)n
   1 (ATGTTTC)n
   1 (ATGTTTG)n
  13 (ATGTTTT)n
   1 (ATGTTTTT)n
2018 (ATT)n
 239 (ATTA)n
  66 (ATTAA)n
   1 (ATTAAAG)n
  13 (ATTAAC)n
   2 (ATTAACA)n
   1 (ATTAACAT)n
   1 (ATTAACT)n
   3 (ATTAAG)n
   3 (ATTAAGA)n
   1 (ATTAAGAT)n
   3 (ATTAAGT)n
  23 (ATTAAT)n
   8 (ATTAATA)n
   2 (ATTAATC)n
   4 (ATTAATG)n
   1 (ATTAATGC)n
   2 (ATTAATGT)n
   4 (ATTAATT)n
  94 (ATTAC)n
  10 (ATTACA)n
   3 (ATTACAA)n
   1 (ATTACAAGC)n
   5 (ATT

  13 (CAGAGC)n
   7 (CAGAGCA)n
   1 (CAGAGCACT)n
   2 (CAGAGCAG)n
   1 (CAGAGCAGAA)n
   1 (CAGAGCAGG)n
   4 (CAGAGCC)n
   1 (CAGAGCCCA)n
   2 (CAGAGCCTAA)n
   1 (CAGAGCT)n
  20 (CAGAGG)n
   7 (CAGAGGA)n
   2 (CAGAGGAA)n
   1 (CAGAGGAG)n
   1 (CAGAGGC)n
   1 (CAGAGGGAAG)n
   1 (CAGAGGGAGA)n
   1 (CAGAGGGATG)n
  27 (CAGAGGT)n
   1 (CAGAGGTG)n
   1 (CAGAGT)n
   1 (CAGAGTA)n
   1 (CAGAGTC)n
   1 (CAGAGTG)n
  25 (CAGAT)n
   5 (CAGATA)n
   7 (CAGATAA)n
   1 (CAGATAAA)n
   1 (CAGATACCA)n
   1 (CAGATACT)n
   1 (CAGATAGA)n
   1 (CAGATAT)n
   7 (CAGATC)n
   2 (CAGATCA)n
   1 (CAGATCC)n
   1 (CAGATCCA)n
   1 (CAGATCCCC)n
   1 (CAGATCCCTA)n
   2 (CAGATG)n
   1 (CAGATGT)n
   1 (CAGATT)n
   3 (CAGATTA)n
   1 (CAGATTCTG)n
   2 (CAGATTT)n
 406 (CAGC)n
  92 (CAGCA)n
  20 (CAGCAA)n
   1 (CAGCAAC)n
   1 (CAGCAACAG)n
   1 (CAGCAATTCC)n
   8 (CAGCAC)n
   7 (CAGCACA)n
   1 (CAGCACAG)n
   1 (CAGCACATCA)n
   1 (CAGCACC)n
   2 (CAGCACCA)n
   2 (CAGCACCTT)n
   1 (CAGCACTAA)n
  17 (CAGCAG)n
   4 (CAGCAGA)n
   3 

...

1834 (TAA)n
 325 (TAAA)n
  39 (TAAAA)n
   7 (TAAAAA)n
   1 (TAAAAAT)n
   2 (TAAAAC)n
   1 (TAAAACACGC)n
   1 (TAAAACTGT)n
   1 (TAAAAG)n
   1 (TAAAAGTTC)n
   1 (TAAAATA)n
  11 (TAAAATG)n
   4 (TAAAC)n
   2 (TAAACA)n
   1 (TAAACATAGC)n
   1 (TAAACATGG)n
   5 (TAAACC)n
   1 (TAAACCAGGA)n
   1 (TAAACCAT)n
   1 (TAAACCCTGT)n
   1 (TAAACTACC)n
   1 (TAAACTCCTG)n
   1 (TAAACTG)n
   2 (TAAAG)n
   1 (TAAAGA)n
   1 (TAAAGACTGA)n
   1 (TAAAGC)n
   1 (TAAAGCGCCA)n
   1 (TAAAGTC)n
   1 (TAAAGTG)n
   6 (TAAAT)n
   1 (TAAATA)n
   1 (TAAATACA)n
   1 (TAAATAG)n
   1 (TAAATAGACA)n
   1 (TAAATAGTGCA)n
   1 (TAAATAT)n
   1 (TAAATCA)n
 190 (TAAATG)n
   1 (TAAATTG)n
 191 (TAAC)n
  29 (TAACA)n
  26 (TAACAA)n
   1 (TAACAAC)n
   1 (TAACAACCCG)n
   1 (TAACAAGC)n
   2 (TAACAAT)n
   6 (TAACAC)n
   2 (TAACACA)n
   1 (TAACACACAC)n
   1 (TAACACAT)n
   1 (TAACACATA)n
   2 (TAACACATTA)n
   1 (TAACACT)n
   1 (TAACACTG)n
   1 (TAACACTGGG)n
  13 (TAACAG)n
   2 (TAACAGA)n
   1 (TAACAGC)n
   1 (TAACAGCAGC)n
   1 (TAACAGGC

38714 (TC)n
1008 (TCA)n
 430 (TCAA)n
  78 (TCAAA)n
   1 (TCAAAAG)n
   1 (TCAAAAGACG)n
   1 (TCAAAGG)n
   3 (TCAAAT)n
   1 (TCAAATA)n
   1 (TCAAATAAC)n
   1 (TCAAATCAAA)n
  17 (TCAAC)n
  13 (TCAACA)n
   1 (TCAACAATAC)n
   1 (TCAACAC)n
   3 (TCAACAT)n
   1 (TCAACATGCT)n
   1 (TCAACATGG)n
   6 (TCAACC)n
   1 (TCAACCC)n
   2 (TCAACCT)n
   1 (TCAACT)n
   1 (TCAACTA)n
   1 (TCAACTATAG)n
   1 (TCAACTGTT)n
   8 (TCAAG)n
   2 (TCAAGA)n
   1 (TCAAGAGAG)n
   1 (TCAAGAGATT)n
   1 (TCAAGCTGG)n
   1 (TCAAGG)n
   1 (TCAAGT)n
   2 (TCAAGTC)n
 236 (TCAAT)n
  14 (TCAATA)n
   5 (TCAATAA)n
   1 (TCAATAC)n
   1 (TCAATAG)n
   6 (TCAATAT)n
   2 (TCAATC)n
   1 (TCAATCAG)n
   1 (TCAATCTT)n
   1 (TCAATCTTG)n
   9 (TCAATG)n
   1 (TCAATGA)n
   1 (TCAATGC)n
   1 (TCAATGCTGG)n
   3 (TCAATGG)n
   1 (TCAATGT)n
  27 (TCAATT)n
   2 (TCAATTA)n
   2 (TCAATTG)n
 496 (TCAC)n
  22 (TCACA)n
   9 (TCACAA)n
   1 (TCACAAC)n
   1 (TCACAAG)n
   1 (TCACAAT)n
  33 (TCACAC)n
   4 (TCACACA)n
   1 (TCACACAAGC)n
   1 (TCACACACTG)n
   1

1233 (TCTCTC)n
  15 (TCTCTCA)n
   1 (TCTCTCAACC)n
   8 (TCTCTCAC)n
   1 (TCTCTCACGT)n
   1 (TCTCTCACTC)n
   1 (TCTCTCACTG)n
   1 (TCTCTCAG)n
   1 (TCTCTCAGTG)n
   2 (TCTCTCAT)n
  91 (TCTCTCC)n
   4 (TCTCTCCA)n
   2 (TCTCTCCAC)n
   1 (TCTCTCCAT)n
   2 (TCTCTCCATG)n
   7 (TCTCTCCC)n
   2 (TCTCTCCCTG)n
   2 (TCTCTCCT)n
   1 (TCTCTCCTC)n
   3 (TCTCTCG)n
   3 (TCTCTCGC)n
   1 (TCTCTCGCAC)n
   6 (TCTCTCGCTC)n
   1 (TCTCTCGT)n
  34 (TCTCTCT)n
   2 (TCTCTCTAG)n
   2 (TCTCTCTAT)n
   1 (TCTCTCTATC)n
   6 (TCTCTCTC)n
   1 (TCTCTCTCC)n
   5 (TCTCTCTCCC)n
   6 (TCTCTCTCGC)n
   1 (TCTCTCTCGG)n
   1 (TCTCTCTCTA)n
  12 (TCTCTCTCTC)n
  15 (TCTCTCTCTG)n
   7 (TCTCTCTCTT)n
  52 (TCTCTCTG)n
   1 (TCTCTCTGAG)n
   1 (TCTCTCTGC)n
   1 (TCTCTCTGG)n
   1 (TCTCTCTGTA)n
  10 (TCTCTCTGTC)n
   2 (TCTCTCTGTG)n
  10 (TCTCTCTT)n
   1 (TCTCTCTTC)n
   1 (TCTCTCTTTT)n
 272 (TCTCTG)n
   2 (TCTCTGA)n
   1 (TCTCTGACTC)n
   1 (TCTCTGAG)n
  10 (TCTCTGC)n
   3 (TCTCTGG)n
   6 (TCTCTGT)n
   1 (TCTCTGTA)n
   1 (TCTCTGTAAC)n
   

1268 (TGTA)n
  70 (TGTAA)n
   7 (TGTAAA)n
   1 (TGTAAAA)n
  13 (TGTAAC)n
   8 (TGTAACA)n
   1 (TGTAACATG)n
   1 (TGTAACC)n
   1 (TGTAACGA)n
   1 (TGTAACT)n
   1 (TGTAACTAT)n
   1 (TGTAACTCATG)n
   1 (TGTAACTGG)n
   4 (TGTAAG)n
   1 (TGTAAGG)n
   1 (TGTAAGTGTG)n
  12 (TGTAAT)n
   9 (TGTAATA)n
   1 (TGTAATAACA)n
   1 (TGTAATAG)n
   1 (TGTAATATAC)n
   1 (TGTAATCCA)n
   3 (TGTAATG)n
   1 (TGTAATGA)n
   1 (TGTAATT)n
   1 (TGTAATTGAA)n
 184 (TGTAC)n
  18 (TGTACA)n
   1 (TGTACAC)n
   1 (TGTACAG)n
   4 (TGTACAT)n
   1 (TGTACATG)n
   2 (TGTACCATGT)n
   1 (TGTACCG)n
   4 (TGTACG)n
   1 (TGTACGG)n
   1 (TGTACGTGTG)n
   5 (TGTACT)n
   1 (TGTACTA)n
   1 (TGTACTATAC)n
   2 (TGTACTC)n
   1 (TGTACTCTAC)n
   9 (TGTACTG)n
   1 (TGTACTGCAT)n
   1 (TGTACTGTAC)n
   2 (TGTACTGTAT)n
   1 (TGTACTGTGG)n
   1 (TGTACTTT)n
  61 (TGTAG)n
   3 (TGTAGA)n
   6 (TGTAGAA)n
   3 (TGTAGAC)n
   1 (TGTAGACAAC)n
   3 (TGTAGAG)n
   1 (TGTAGAGCC)n
   3 (TGTAGAT)n
   9 (TGTAGC)n
   1 (TGTAGCA)n
   1 (TGTAGCAA)n
   1 (TGTAGCAG)

11882 A-rich
5626 DIRS/DIRS
252474 DNA
17686 DNA/CMC-EnSpm
13025 DNA/Crypton
 580 DNA/Dada
4819 DNA/Ginger
6323 DNA/Harbinger
8407 DNA/IS3EU
2374 DNA/Kolobok
1345 DNA/Maverick
 517 DNA/PIF-ISL2EU
13780 DNA/PiggyBac
6055 DNA/Sola
634644 DNA/TcMar
217856 DNA/hAT
11630 G-rich
67161 GA-rich
1531 LINE/CR1
168485 LINE/Crack
6570 LINE/Jockey
29550 LINE/L1
107372 LINE/L2
11590 LINE/Nimb
14605 LINE/Penelope
 214 LINE/R2
 684 LINE/RTE
50058 LINE/RTEX
232509 LINE/Rex1
 988 LINE/Togen
30774 LINE/Tx1
74915 LTR
5098 LTR/BEL
9825 LTR/Copia
11538 LTR/ERV
16553 LTR/ERV1
95012 LTR/Gypsy
5837 SINE/Deu
104180 SINE2/tRNA
45137 Satellite
5501 Simple_repeat
1008726 Unknown
5735 Unknown/Unknown
 116 polypurine


In [19]:
!cat /Volumes/web/metacarcinus/Salmo_Calig/GENOMES/v2/RefSeq/ssal_repeats_v5.1.TEannotations.noNW.gff |\
awk -F"\t" '{print $9}' |\
awk -F" " '{print $2}' |\
sort | uniq -c

755060 
5626 DIRS/DIRS
252474 DNA
17686 DNA/CMC-EnSpm
13025 DNA/Crypton
 580 DNA/Dada
4819 DNA/Ginger
6323 DNA/Harbinger
8407 DNA/IS3EU
2374 DNA/Kolobok
1345 DNA/Maverick
 517 DNA/PIF-ISL2EU
13780 DNA/PiggyBac
6055 DNA/Sola
634644 DNA/TcMar
217856 DNA/hAT
1531 LINE/CR1
168485 LINE/Crack
6570 LINE/Jockey
29550 LINE/L1
107372 LINE/L2
11590 LINE/Nimb
14605 LINE/Penelope
 214 LINE/R2
 684 LINE/RTE
50058 LINE/RTEX
232509 LINE/Rex1
 988 LINE/Togen
30774 LINE/Tx1
74915 LTR
5098 LTR/BEL
9825 LTR/Copia
11538 LTR/ERV
16553 LTR/ERV1
95012 LTR/Gypsy
5837 SINE/Deu
104180 SINE2/tRNA
45137 Satellite
5501 Simple_repeat
1008726 Unknown
5735 Unknown/Unknown
